### 方針
 ※ 使用データ...頂いた小麦のデータ
1. 適切な塩基の組み合わせを選ぶ（特徴量選択）
2. １で選んだ転写因子の組み合わせを選んだときにもっともエネルギーが低くなるような関数を設定する
3. 表現型(収穫量)の要素を一つずつ変化させる。この前後でエネルギーがどのくらい変わったのか確認する（比）。
4. 乱数(0 ≦ R ≦ 1の一様乱数)より高かったら採択、低かったら棄却
5. ３に戻る
6. ４に100回たどり着くたびにlistに記録


### 仮定
- 小麦データの収量行列は１行４列。
- 候補として最大値、最長値、平均値を考えたが、今回は0列目を使うことにする（よう言われました）

### 1. 特徴量選択
(考え)変数間の相互作用を考えたほうが良いのだろう、反復特徴量選択は時間がかかる→モデルベース特徴量選択 <br/>
(結論)1279塩基から640塩基が残った

In [ ]:
説明変数...pd.DataFrame に変換できる何か ( DataFrame 自体, 辞書など) を渡す
目的変数...列名の文字列、もしくは pd.Series に変換できる何か ( Series、もしくはリストlike )を渡す

In [1]:
# 説明変数
import pandas as pd
data_SNP = pd.read_table('CIMMYTwheat_markers.txt', header=None,  sep =  ' ')
print('(SNPサンプル数, 塩基数)=', data_SNP.shape)
#print(data_SNP.style)
#print('SNPサンプル数：', len(data_SNP))
#print(data_SNP.head)

(SNPサンプル数, 塩基数)= (599, 1279)


/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
# 目的変数
data_yields = pd.read_table('CIMMYTwheat_yields.txt', header=None,  sep =  ' ')
print('SNPサンプル数：', len(data_yields))
print(type(data_yields))
print(data_yields.shape)
print(data_yields.iloc[0]) #0行目
print(data_yields.iloc[0, 0]) #[0, 0]

'''
data_amount_list = []
for i in range(len(data_yields)):
    canditate = data_yields.iloc[i, 0]
    for j in range(1, 4):
        if data_yields.iloc[i, j] >= canditate:
            canditate = data_yields.iloc[i, j]
    data_amount_list.append(canditate)
'''

data_amount_list = []
for i in range(len(data_yields)):
    data_amount_list.append(data_yields.iloc[i, 0])

#print(data_amount)
print(len(data_amount_list))
print(type(data_amount_list))
print(data_amount_list[3])

SNPサンプル数： 599
<class 'pandas.core.frame.DataFrame'>
(599, 4)
0    1.671629
1   -1.727470
2   -1.890285
3    0.050916
Name: 0, dtype: float64
1.671629
599
<class 'list'>
0.7854395000000001


/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [79]:
# list→seriesへ変換
import pandas as pd
l_1d = [0, 1, 2]
s = pd.Series(l_1d)
print(s)

0    0
1    1
2    2
dtype: int64


In [102]:
data_amount = pd.Series(data_amount_list)

In [103]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_SNP, data_amount, test_size=0.3) #x, y ,test0.3, train0.7

In [104]:
print(y_train.ndim)
print(y_train.shape)
#y_train = y_train.values.ravel()
#print(y_train.ndim)
#print(len(y_train))

1
(419,)


In [105]:
print(y_test.ndim)
print(y_test.shape)
#y_test = y_test.values.ravel()
#print(y_test.ndim)
#print(len(y_test))

1
(180,)


In [ ]:
'''
lab_enc = preprocessing.LabelEncoder()
train_scores_encoded = lab_enc.fit_transform(y_train)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(train_scores_encoded))
'''

In [107]:
'''
#単変量統計
from sklearn.feature_selection import SelectPercentile
select = SelectPercentile(percentile = 40)
select.fit(x_train, y_train)

# 訓練セットの変換
x_train_selected = select.transform(x_train)

print('x_train_shape : {}'. format(x_train.shape))
print('x_train_selected.shape: {}'. format(x_train_selected.shape))
'''

# モデルベース特徴量選択（embedded method、組み込み法）
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

# estimator として RandomForestRegressor を使用。重要度が median 以上のものを選択
select = SelectFromModel(RandomForestRegressor(n_estimators = 100, random_state = 42), threshold = 'median')
select.fit(x_train, y_train)

#yはselect.fitしなくていい
x_train_selected = select.transform(x_train)
x_test_selected = select.transform(x_test)

print('x_train.shape: {}'.format(x_train.shape))
print('x_train_l1_shape: {}'.format(x_train_selected.shape))

mask = select.get_support()
print(mask)
print(type(mask))


#import matplotlib.pyplot as plt
#plt.matshow(select.get_support().reshape(1, -1), cmap="gray_r")
#plt.xlabel("sample index")
#plt.yticks(())



x_train.shape: (419, 1279)
x_train_l1_shape: (419, 640)
[ True  True  True ...  True  True False]
<class 'numpy.ndarray'>


In [108]:
from sklearn import preprocessing, utils

lab_enc = preprocessing.LabelEncoder()

train_scores_encoded = lab_enc.fit_transform(y_train)
#print(train_scores_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(train_scores_encoded))

continuous
multiclass
multiclass


In [109]:
test_scores_encoded = lab_enc.fit_transform(y_test)
#print(test_scores_encoded)
print(utils.multiclass.type_of_target(y_test))
print(utils.multiclass.type_of_target(y_test.astype('int')))
print(utils.multiclass.type_of_target(test_scores_encoded))

continuous
multiclass
multiclass


In [2]:
'''
#モデルベース特徴量選択を用いた結果
from sklearn.linear_model import LogisticRegression
score = LogisticRegression().fit(x_train_selected, train_scores_encoded).score(x_test_selected, test_scores_encoded)
print(f"x_train_selected.shape : {x_train_selected.shape}")
#print(f'Score with only selected features : {score:.3f}')


#score = LogisticRegression().fit(x_train_l1, y_train).score(x_test_selected, y_test)
#print("Test score: {:.3f}".format(score))  # 0と出る
'''

'\n#モデルベース特徴量選択を用いた結果\nfrom sklearn.linear_model import LogisticRegression\nscore = LogisticRegression().fit(x_train_selected, train_scores_encoded).score(x_test_selected, test_scores_encoded)\nprint(f"x_train_selected.shape : {x_train_selected.shape}")\n#print(f\'Score with only selected features : {score:.3f}\')\n\n\n#score = LogisticRegression().fit(x_train_l1, y_train).score(x_test_selected, y_test)\n#print("Test score: {:.3f}".format(score))  # 0と出る\n'

In [113]:
print(mask)
print(type(mask)) #numpy.ndarray
print(len(mask)) #もとの特徴量の数

[ True  True  True ...  True  True False]
<class 'numpy.ndarray'>
1279


In [12]:
 #True = 1, False = 0
print(True + True)
print(True * 10)

2
10


###  確率場

- 確率場$Z$がギブス的である ⇔ そのときに確率密度関数が👇となっている
$$
f(z) =  \frac{e^{-E(z)}}{Z}
$$
<br/>
- エネルギー関数$E$をもつギブス分布（or ボルツマン分布）= 👆に対応して定まる分布
- $Z$は定数

### 2. エネルギー関数の設定
- $E()$の設定→元祖maskが一番小さくなるように、相互情報量ではmask同士が一番大きくなる
- 確率をどう書くか→$f(z)$の設定、定数は無視
- エネルギーの大きさ比較→比を計算
- 乱数発生→uniform...
- 採択、棄却→更新するか否かの決断、エネルギーが一つ前のより小さかったら採択

In [21]:
from sklearn.metrics.cluster import normalized_mutual_info_score
print(normalized_mutual_info_score([0, 0, 1, 1], [0, 0, 1, 1])) # 最大1.0
print(normalized_mutual_info_score([0, 0, 0, 0], [0, 1, 2, 3])) # 最小0.0
print(normalized_mutual_info_score([0, 0, 1, 1], [1, 1, 0, 0])) # 違うリストでも1.0をマークすることがある

1.0
0.0
1.0


/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


In [37]:
# appendでも listにするかndarrayにするかで書き方異なる

l = list(range(3))
print(l)

l.append(100)
print(l)

import numpy  as np
a = np.arange(12)
print(a)
np.append(a, [6, 4, 2])
print(a)

[0, 1, 2]
[0, 1, 2, 100]
[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [125]:
E = [] #Eは配列、どんどん入れていく(一つ後ろで計算の際使用される)
def Energy(mask_trial):
    return 1-normalized_mutual_info_score(mask_trial, mask) #エネルギーの定義≡1-標準化相互情報量 とした
#np.append(E, Energy(mask)) #第一要素を追加
E.append(Energy(mask))
#print(Energy(mask))
#print(mask)
print(type(E))
print(E[0])

<class 'list'>
0.0


/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


In [118]:
new_y = data_amount #data_amountはSeries
new_y[3]

0.7854395000000001

In [ ]:
# future warning : Normalized Mutual Information (NMI)

In [128]:
#mask_trial ...new_yによって選ばれた塩基mask(x)、forループを回すたび値が変わる、maskと比較するためにあるだけなので保存する必要なし

save_y = [] #ほしいベクトルを貯める配列
new_y = data_amount #以降要素を一つずつ変えていく #data_amountはSeries

import math

for i in range(1, 100):   
    new_y[i] = 1 - new_y[i] #要素一つだけ変えた
    
    select = SelectFromModel(RandomForestRegressor(n_estimators = 100, random_state = 42), threshold = 'median', max_features=640)
    select.fit(data_SNP, new_y)
    data_SNP_l1 = select.transform(data_SNP)
    mask_trial = select.get_support()

    #np.append(E, Energy(mask_trial))  #mask_trialは一つのndarrayの中で順番に要素を更新していく
    E.append(Energy(mask_trial))
    r = math.exp(-E[i])/math.exp(-E[i-1])
    R = np.random.uniform(0.0, 1.0)
    if R >= r : #f(z)が一個前のものよりも小さかったら⇔E[i]が一個前のものよりも大きかったら
        new_y[i] =1 - new_y[i] #元に戻す   
    if i%100 == 0:
        save_y.append(new_y)
    else:
            continue
    if len(save_y) > 1:
        break

/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_metho

/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_metho

/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_metho

/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


In [130]:
print(len(save_y))
print(save_y)

0
[]
